In [99]:
%pip install langchain langchain-openai langchainhub --quiet

Note: you may need to restart the kernel to use updated packages.


# example from https://python.langchain.com/v0.1/docs/modules/agents/agent_types/react/

In [100]:
from langchain import hub
from langchain.agents import AgentExecutor, create_react_agent
from langchain_openai import OpenAI

In [101]:
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/react")

from pprint import pprint
pprint(prompt.template)
#a=prompt.format_prompt(context="some context",question="some question")
#for msg in a.messages:
#    print(msg.content)

('Answer the following questions as best you can. You have access to the '
 'following tools:\n'
 '\n'
 '{tools}\n'
 '\n'
 'Use the following format:\n'
 '\n'
 'Question: the input question you must answer\n'
 'Thought: you should always think about what to do\n'
 'Action: the action to take, should be one of [{tool_names}]\n'
 'Action Input: the input to the action\n'
 'Observation: the result of the action\n'
 '... (this Thought/Action/Action Input/Observation can repeat N times)\n'
 'Thought: I now know the final answer\n'
 'Final Answer: the final answer to the original input question\n'
 '\n'
 'Begin!\n'
 '\n'
 'Question: {input}\n'
 'Thought:{agent_scratchpad}')


In [102]:


# Choose the LLM to use
#llm = OpenAI()
from pretty_print_callback_handler import PrettyPrintCallbackHandler
callback = PrettyPrintCallbackHandler()
llm = OpenAI(streaming=False, temperature=0 , callbacks=[callback])



In [103]:
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool


tools = []
class BashValidatorInput(BaseModel):
    code: str = Field(description="The code to validate")

def bash_validate(code: str) -> str:
    """Check if the code is valid bash code"""

    ## Write code to a tempfile and run it
    import tempfile
    import subprocess
    import os
    
    with tempfile.NamedTemporaryFile(mode='w', delete=False) as f:
        f.write(code)
        temp_file = f.name

        try:
            result=subprocess.run(["bash","-n",temp_file], check=True)
            if (result.returncode != 0):
                return "The code is not valid bash code"
            else:
                return "The code is valid bash code"
        except subprocess.CalledProcessError as e:
            return f"The code is not valid bash code\n{e}"
        finally:
            print("removing temp file " + temp_file)
            os.remove(temp_file)

bash_validator_tool = StructuredTool.from_function(
    func=bash_validate,
    name="bash_validator",
    description="Check if the code is valid bash code",
    args_schema=BashValidatorInput,
  #  return_direct=True,
    handle_tool_eror=True
)


class MarkdownValidatorInput(BaseModel):
    code: str = Field(description="The code to validate")

def markdown_validate(code: str) -> str:
    """Check if the code is valid markdown"""
    if (code.startswith("```bash") and code.endswith("```")):
        return "The code is valid markdown"
    else:
        return "The code is not valid markdown code block"

markdown_validator_tool = StructuredTool.from_function(
    func=markdown_validate,
    name="markdown_validator",
    description="Check if the code is valid markdown",
    args_schema=MarkdownValidatorInput,
  #  return_direct=True,
    handle_tool_eror=True
)


tools.append(bash_validator_tool)
tools.append(markdown_validator_tool)

In [104]:

# Construct the ReAct agent
agent = create_react_agent(llm, tools, prompt)

In [105]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools,
         verbose=True,return_intermediate_steps=True,
           callbacks=[callback], max_iterations=15)

In [106]:
agent_executor.invoke({"input": "Write me a shell script to list the files in the current directory. Output the script as markdown bash code block"})



[chain][start] - inputs {'input': 'Write me a shell script to list the files in the current directory. Output the script as markdown bash code block'}


> Entering new AgentExecutor chain...

[llm][start] - prompts: Answer the following questions as best you can. You have access to the following tools:

bash_validator: bash_validator(code: str) -> str - Check if the code is valid bash code
markdown_validator: markdown_validator(code: str) -> str - Check if the code is valid markdown

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [bash_validator, markdown_validator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Write me a shell script to list the files i

{'input': 'Write me a shell script to list the files in the current directory. Output the script as markdown bash code block',
 'output': '```bash\nls\n```',
 'intermediate_steps': [(AgentAction(tool='bash_validator', tool_input='ls', log=' I should use bash_validator to check if the code is valid bash code and markdown_validator to check if the code is valid markdown\nAction: bash_validator\nAction Input: "ls"'),
   'The code is valid bash code'),
  (AgentAction(tool='markdown_validator', tool_input='ls', log='I should now use markdown_validator to check if the code is valid markdown\nAction: markdown_validator\nAction Input: "ls"'),
   'The code is not valid markdown code block'),
  (AgentAction(tool='markdown_validator', tool_input='```bash ls ```', log=' I should wrap the code in a markdown code block\nAction: markdown_validator\nAction Input: "```bash ls ```"'),
   'The code is valid markdown')]}